In [ ]:
import sys
import os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from deep_convnet import DeepConvNet
from dataset.mnist import load_mnist

# 8.1 더 깊게
# 8.1.1 더 깊은 신경망으로
"""
딥러닝 : 층을 깊게 한 심층 신경망

MNIST를 학습하는 다음과 같은 신경망을 구성한다.
C : Conv, R : ReLU, P : Pooling, A : Affine, D : Dropout, S : Softmax
→ C16→R→C16→R→P → C32→R→C32→R→P → C64→R→C64→R→P → A→R→D → A→D→S →
C 계층의 필터는 3*3, 숫자는 채널의 수
풀링 계층을 추가하여 중간 데이터의 공간 크기를 줄여간다. 마지막 완전연결계층에서는 드롭아웃을 사용
가중치 초깃값은 He, 가중치 매개변수 갱신에는 Adam을 사용.
구현 코드는 deep_convnet.py 참고, 훈련 코드는 deepnet.py 참고.
"""

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()
# 학습에 긴 시간이 걸리므로 미리 학습한 결과를 pkl로 배포
network.load_params('deep_convnet_params.pkl')

print(network.accuracy(x_test, t_test))  # 0.9935
# 99.35% 인식률을 보인다. 어떤 이미지를 인식하지 못했는지는 misclassified_mnist.py 참고

# 8.1.2 정확도를 더 높이려면
"""
NOTE : MNIST는 상대적으로 단순하기 때문에 층을 깊게 하지 않고도 좋은 결과가 나오지만
대규모 일반 사물 인식에 대해서는 층을 깊게 하면 얻는 정확도가 크다.

데이터 확장data augmentation : 훈련 이미지를 알고리즘을 이용해 인위적으로 늘린다.
이미지 회전, 이동, crop, flip, 확대, 축소, 밝기 등을 변경한다.
"""

# 8.1.3 깊게 하는 이유
"""
층의 깊이에 비례해 정확도가 높아지는 경향이 있다.
층을 깊게하면 신경망의 매개변수 수가 줄어들며, 적은 매개변수로 같은 수준의 표현력을 얻을 수 있다.
예를 들어, 5*5 필터 합성곱 연산 1회는 3*3 합성곱 연산 2회로 대체할 수 있다.
하지만 전자의 매개변수는 25개(5*5)이지만 후자는 18개(3*3*2)이다.

층을 깊게하며 중간에 활성화 함수를 끼우는 것도 표현력이 개선된다. 활성화 함수가 신경망에
'비선형' 힘을 가하기 때문이다.

깊은 신경망은 학습할 문제를 계층적으로 분해할 수 있기 때문에 적은 데이터로 학습 가능하다.
"""

# 8.2 딥러닝의 초기 역사
"""
딥러닝이 주목받은 계기는 이미지 인식 대회 ILSVRC의 2012년 대회이다.
AlexNet이 기존 대비 압도적 성적(오류율 25.8%->16.4%)으로 우승한 이후 딥러닝이 주류가 됐다.
"""

# 8.2.1 이미지넷
"""
100만장이 넘는 이미지를 담고 있는 데이터셋. ILSVRC의 시험 항목이다.
2015년 150층이 넘는 신경망인 ResNet이 오류율 3.5%로 인간 인식 능력을 넘어섰다.
"""

# 8.2.2 VGG
"""
합성곱 계층과 풀링 계층으로 구성되는 기본적인 CNN. 16층(19층) 신경망을 사용한다.
2014년 대회 2위에 올랐으며 구성이 간단해 응용하기 좋아 많이 사용된다.
"""

# 8.2.3 GoogLeNet
"""
깊이 뿐 아니라 폭의 너비도 깊다는 것이 특징(인셉션 구조) 크기가 다른 여러 필터와 풀링을
필터로 결합한다.
"""

# 8.2.4 ResNet
"""
Residual Netwrk 마이크로소프트가 개발한 네트워크
층이 지나치게 깊으면 학습이 잘 되지 않고 성능이 떨어지는 문제를 해결하기 위해 VGG 신경망 기반에
스킵 연결을 도입했다. 입력 데이터를 일부 합성곱 계층을 건너뛰어 출력에 바로 더한다.
역전파 때 신호 감쇠를 막아준다.

NOTE : 전이 학습transfer learning : 학습된 가중치를 다른 신경망에 복사한 다음
재학습을 수행하는 것. 보유한 데이터셋이 적을 때 유용하다.
"""

# 8.3 더 빠르게(딥러닝 고속화)
"""
과거에는 CPU가 계산을 담당했으나 현재는 복수의 GPU로 분산 수행한다.
"""

# 8.3.1 풀어야 할 숙제
"""
AlexNet의 순전파에서 각 층이 소비하는 시간은 GPU의 95%, CPU의 89%가 합성곱 계층이다.
결국 단일 곱셈-누산을 어떻게 효율적으로 계산하느냐는 문제가 된다.
"""

# 8.3.2 GPU를 활용한 고속화
"""
GPU는 병렬 수치 연산을 고속으로 처리할 수 있으므로 범용 수치 연산에 사용하는 것을 GPU 컴퓨팅
이라고 한다.
CPU 40일 계산이 GPU 6일로 단축하며, cuDNN등 딥러닝 라이브러리를 이용하면 더 빨라진다.
엔비디아와 AMD 중 주로 CUDA를 사용한 엔비디아가 딥러닝에 더 최적화되어 있다.
"""

# 8.3.3 분산 학습
"""
1회 학습에 걸리는 시간을 단축하기 위해 학습을 수평확장하기 위해 다수의 GPU와 컴퓨터에 분산한다.
구글의 텐서플로우와 MS의 CNTK 등이 분산 학습을 지원하는 프레임워크다.
GPU 100개를 사용하면 1개일 때 보다 56배 빨라졌다.
"""

# 8.3.4 연산 정밀도와 비트 줄이기
"""
계산 능력 외에도 메모리 용량과 버스 대역폭 등이 고속화에 병목이 될 수 있다.
컴퓨터는 주로 64비트 배정밀도나 32비트 단정밀도 부동소수점을 사용해 실수를 표현하지만
딥러닝은 높은 수치 정밀도를 요구하지 않기 때문에(신경망의 견고성),
데이터를 퇴화시켜도 출력에 주는 영향이 적다. 16비트 반정밀도만 사용해도 문제가 없다고 알려져있다.

실수의 정밀도에 따른 결과 비교는 half_float_network.py 참고
caluculate accuracy (float64) ...
0.9935
caluculate accuracy (float16) ...
0.9935

최근에는 가중치와 중간 데이터를 1비트로 표현하는 Binarized Neural Networks라는 방법도
등장했다.
"""

# 8.4 딥러닝의 활용
# 8.4.1 사물 검출
"""
사물의 위치와 다수의 사물이 존재할 수 있기 때문에 사물 인식보다 어려운 문제.
R-CNN(Regions with CNN)이 유명하다. 후보 영역 추출 후 각 영역에 CNN을 적용하여 분류한다.
후보 영역 추출은 기존 컴퓨터 비전 분야의 기법을 사용할 수 있고 이 부분까지 CNN으로 처리하는
Faster R-CNN 기법도 있다.
"""

# 8.4.2 분할
"""
이미지를 픽셀 수준에서 분류하는 문제.
이미지 단위가 아니라 픽셀 단위로 추론해야되기 때문에 긴 시간이 걸린다.
낭비를 줄여주는 기법으로 한 번의 순전파 처리로 모든 픽셀을 분류해주는
FCN(Fully Convolutional Network)이 고안되었다.

일반적인 CNN이 완전연결 계층을 이용하는 반면 FCN에서는 이를 '같은 기능을 하는 합성곱 계층'으로
바꾼다. 마지막 출력까지 공간 볼륨을 유지한 채 처리할 수 있다.

NOTE : 입력 크기가 32*10*10인 데이터의 완전 연결 계층은 필터 크기가 32*10*10인
합성곱 계층으로 대체할 수 있다.
완전연결 계층의 출력 노드가 100개라면 이러한 필터 100개를 사용하면 된다.
"""

# 8.4.3 사진 캡션 생성
"""
NIC(Neural Image Caption) 모델이 대표적. 심층 CNN과 자연어를 다루는 순환 신경망(RNN)으로
구성된다.
멀티모달 처리 : 사진이나 자연어 같은 여러 종류의 정보를 조합하고 처리하는 것.abs

NOTE : RNN은 신경망이 순환적 네트워크 구조를 가진다. 이로 인해 이전에 생성한 정보에 영향을
받는 점이 RNN의 특징이다. 자연어와 시계열 데이터 등 연속성 있는 데이터를 다룰 때 사용한다.
"""

# 8.5 딥러닝의 미래
# 8.5.1 이미지 스타일 변환
"""
스타일 이미지와 콘테츠 이미지를 결합해 새로운 이미지를 생성한다.
"""

# 8.5.2 이미지 생성
"""
이미지 입력 없이 새로운 이미지를 그려내는 연구.
DCGAN(Deep Convolutional Generative Adversarial Network) 기법을 통해 학습 데이터에는
존재하지 않는 새로운 침실 이미지들을 생성했다.

DCGAN은 생성자Generator와 식별자Discriminator라 불리는 2개의 신경망을 이용한다.
생성자가 이미지를 생성하고 식별자는 진짜인지를 판별한다. 둘을 겨루도록 학습시켜 생성자는
더 정교한 이미지 생성을, 식별자는 더 정확한 판별을 하게 된다.
이런 개념을 GAN(Generative Adversarial Network)이라 한다.

NOTE : 이전까지의 기계학습은 지도 학습supervised learning이라는 유형의 문제였다.
하지만 이번 절의 문제는 지도용 데이터는 주어지지 않고 대량의 이미지만 주어진다.
즉, 지도 없이 스스로 학습하는 자율 합스unsupervised learning문제이다.
"""

# 8.5.3 자율 주행
"""
경로 계획과 센서 기술 등이 있지만 가장 중요한 기술은 주위 환경을 올바르게 인식하는 것.
SegNet이라는 CNN기반 신경망은 입력 이미지를 픽셀 수준에서 인식할 수 있다.
"""

# 8.5.4 Deep Q-Network(강화학습)
"""
강화학습reinforcement learning : 가르침에 의존하는 지도 학습과는 다른게 시행착오를 통해
스스로 학습하게 하는 시도.
에이전트는 환경에 맞게 행동을 선택하고 행동에 의해 환경이 변화한다. 환경이 변화하면 에이전트는
어떤 보상을 얻는다. 강화학습의 목적은 더 나은 보상을 받는 쪽으로 에이전트의 지침을 설정하는 것이다.

행동의 보상은 항상 명확한 것이 아니기 때문에 게임 점수나 게임 종료 등 명확한 지표로부터
역산해 예상 보상을 정해야 한다.

DQN : Q학습이라는 강화학습 알고리즘을 기초로 한다. Q학습은 최적 행동 가치 함수로 행동을
결정하는데 이를 CNN으로 흉내낸다.
"""
